In [ ]:
# install dependecies

!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
# mount google drive to fetch the relevant files

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
# define the path of the created dataset

created_data_path = "/content/drive/MyDrive/Dataset/created_data/final_dataset/"

# Inference code

## Steps

1. Load the trained model
2. Load the feature extractor
3. Fetch the audio sample
4. Sample the audio using a defined sampling rate (16K in our case)
5. Add a batch dimension
6. Extract features and feed into the model
7. **Magic...**
8. Get the output and decode it


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor
import librosa
import torch

# Load model and processor from the specified folder
model_folder_path = "/content/drive/MyDrive/Dataset/models/whisper-small-en-final_dataset"
model = WhisperForConditionalGeneration.from_pretrained(model_folder_path)

# Initialize processor and feature extractor
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language='English', task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

# Load audio file
audio_file_path = "/content/drive/MyDrive/Dataset/M/M02/Session2/wav_arrayMic/0232.wav"
# normal_audio_path ="/content/drive/MyDrive/Dataset/MC/MC01/Session2/wav_arrayMic/0118.wav"
audio, sample_rate = librosa.load(audio_file_path, sr=16000)

# Extract audio features
inputs = feature_extractor(audio, sampling_rate=sample_rate, return_tensors="pt")

# Preprocess audio data
inputs = processor.feature_extractor.pad(inputs, return_tensors="pt").input_features[0]

# Initialize model
inputs = torch.tensor(inputs).unsqueeze(0)

# Perform inference
outputs=model.generate(inputs)

# Decode transcribed text
transcribed_text = processor.batch_decode(outputs, skip_special_tokens=True)
print("Transcribed Text:", transcribed_text)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribed Text: ['a good attitude is unbeatable']


**Whisper model without fine tuning**

In [ ]:
 from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor
import librosa
import torch
from transformers import WhisperForConditionalGeneration

original_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Initialize processor and feature extractor
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language='English', task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

# Load audio file
audio_file_path = "/content/drive/MyDrive/Dataset/M/M02/Session2/wav_arrayMic/0232.wav"
#normal_audio_path ="/content/drive/MyDrive/Dataset/MC/MC01/Session2/wav_arrayMic/0118.wav"
audio, sample_rate = librosa.load(audio_file_path, sr=16000)

# Extract audio features
inputs = feature_extractor(audio, sampling_rate=sample_rate, return_tensors="pt")

# Preprocess audio data
inputs = processor.feature_extractor.pad(inputs, return_tensors="pt").input_features[0]

# Initialize model
inputs = torch.tensor(inputs).unsqueeze(0)

# Perform inference
outputs=original_model.generate(inputs)

# Decode transcribed text
transcribed_text = processor.batch_decode(outputs, skip_special_tokens=True)
print("Transcribed Text:", transcribed_text)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcribed Text: [" I'm good at this, because I'm good at this, because I'm beautiful."]
